In [1]:
#Tensorflow Import and GPU recognition
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9429721708880714195, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7610448864307365526
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4677164607111486904
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15372070743028525326
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!unzip '/content/drive/My Drive/Progetto Advanced Machine Learning/fruits.zip' -d '/content'

Streaming output truncated to the last 5000 lines.
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_16_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_17_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_181_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_182_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_183_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_184_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_185_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_186_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_187_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_188_100.jpg  
  inflating: /content/fruits-360_dataset/fruits-360/Training/Tangelo/r_189_100.jpg  
  inflating: /co

In [5]:
#Importing Keras and other useful libs
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from glob import glob
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [6]:
#Defining dataset path and saving it for future usage
path = '/content/fruits-360_dataset/fruits-360'
os.listdir(path)

['Test', 'papers', 'test-multiple_fruits', 'readme.md', 'LICENSE', 'Training']

In [7]:
#Number of pictures in Training folder
training_files = glob(os.path.join(path,'Training', '*/*.jpg'))
image_num = len(training_files)
print("Number of Images: ",image_num)

Number of Images:  60498


In [8]:
#Number of pictures in Test Folder
testing_files = glob(os.path.join(path, 'Test', '*/*.jpg'))
img_num = len(testing_files)
print("Number of Images: ", img_num)

Number of Images:  20622


In [9]:
#Print category details
image_count = []
class_names = []
print('{:18s}'.format('Class'), end='')
print('Count:')
print('-'*24)
for folder in os.listdir(os.path.join(path,'Training')):
  folder_count = len(os.listdir(os.path.join(path,'Training',folder)))
  image_count.append(folder_count)
  class_names.append(folder)
  print('{:20s}'.format(folder), end='')
  print(folder_count)
print('-'*24)
print('Number of Classes:', len(class_names))
print('Average number of images per Class: ', np.array(image_count).mean())

Class             Count:
------------------------
Mango               490
Onion Red           450
Cherry Rainier      738
Lemon               492
Walnut              735
Limes               490
Nut Forest          218
Cactus fruit        490
Pineapple Mini      493
Grapefruit White    492
Raspberry           490
Tomato Cherry Red   492
Apple Golden 2      492
Maracuja            490
Melon Piel de Sapo  738
Grape White 3       492
Quince              490
Onion White         438
Grape White 4       471
Grapefruit Pink     490
Potato White        450
Grape White         490
Carambula           490
Clementine          490
Cauliflower         702
Tamarillo           490
Eggplant            468
Apple Red Yellow 2  672
Banana              490
Lemon Meyer         490
Apple Red Yellow 1  492
Guava               490
Pear Abate          490
Strawberry Wedge    738
Plum                447
Apple Golden 1      492
Cherry 1            492
Tomato 4            479
Pepper Yellow       666
Cocos         

In [10]:
#Definitive paths
train_out_path = os.path.join(path,'Training')
test_out_path = os.path.join(path, 'Test')
print(train_out_path)
print(test_out_path)

/content/fruits-360_dataset/fruits-360/Training
/content/fruits-360_dataset/fruits-360/Test


In [0]:
#2 Approach: Inception v3 pre-trained
from keras_applications.vgg16 import VGG16
from keras_applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from keras import optimizers
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras import regularizers
from keras.layers.convolutional import *
from keras.models import Model

In [0]:
#Implementing some augmentation to avoid overfitting on the training generator
train_datagenerator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, validation_split=0.2, data_format='channels_last')
train_and_val_generator = ImageDataGenerator(rescale=1. / 255, rotation_range=30, zoom_range=0.2, horizontal_flip=True, data_format='channels_last')
test_datagenerator = ImageDataGenerator(rescale= 1./255, data_format='channels_last')

In [36]:
#Creating Batches
image_size = (256, 256)
train_batches = train_datagenerator.flow_from_directory(train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='training', seed = 20052020)
val_batches = train_datagenerator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, subset='validation', seed = 20052020)
train_val_batches = train_and_val_generator.flow_from_directory(directory=train_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, seed = 20052020)
test_batches = test_datagenerator.flow_from_directory(directory=test_out_path, target_size=image_size, color_mode="rgb", class_mode="categorical" ,  batch_size=32, shuffle=False)

Found 48431 images belonging to 120 classes.
Found 12067 images belonging to 120 classes.
Found 60498 images belonging to 120 classes.
Found 20622 images belonging to 120 classes.


In [31]:
#build custom model
model = tf.keras.models.Sequential()
model.add( tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='same', data_format='channels_last', activation='relu', name='conv_1', input_shape=image_size+(3,)))
model.add( tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=1, data_format='channels_last', name='pool_1'))
model.add( tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', data_format='channels_last', activation='relu', name='conv_2'))
model.add( tf.keras.layers.GlobalMaxPool2D(data_format='channels_last', name='pool_2'))
model.add( tf.keras.layers.Dense(120, activation='softmax', name='prediction'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 256, 256, 128)     3584      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 255, 255, 128)     0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 255, 255, 64)      73792     
_________________________________________________________________
pool_2 (GlobalMaxPooling2D)  (None, 64)                0         
_________________________________________________________________
prediction (Dense)           (None, 120)               7800      
Total params: 85,176
Trainable params: 85,176
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Training and Validating the model: Round 1
wait = 1
epochs = 15
model_name='Inception v3'
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=wait)

In [37]:
trainingMdl = model.fit_generator(train_val_batches, epochs=10, verbose=1, shuffle=True, validation_data=val_batches)

Epoch 1/10
1891/1891 [==============================] - 1700s 899ms/step - loss: 1.5882 - accuracy: 0.5819 - val_loss: 0.5057 - val_accuracy: 0.8436
Epoch 2/10
1891/1891 [==============================] - 1708s 903ms/step - loss: 0.3353 - accuracy: 0.8905 - val_loss: 0.2544 - val_accuracy: 0.9146
Epoch 3/10
1891/1891 [==============================] - 1707s 903ms/step - loss: 0.2015 - accuracy: 0.9331 - val_loss: 0.1671 - val_accuracy: 0.9408
Epoch 4/10
1891/1891 [==============================] - 1715s 907ms/step - loss: 0.1487 - accuracy: 0.9488 - val_loss: 0.1099 - val_accuracy: 0.9669
Epoch 5/10
1891/1891 [==============================] - 1709s 904ms/step - loss: 0.1205 - accuracy: 0.9579 - val_loss: 0.1044 - val_accuracy: 0.9611
Epoch 6/10
1891/1891 [==============================] - 1709s 904ms/step - loss: 0.0977 - accuracy: 0.9665 - val_loss: 0.1092 - val_accuracy: 0.9665
Epoch 7/10
1891/1891 [==============================] - 1721s 910ms/step - loss: 0.0871 - accuracy: 0.9706

In [96]:
import sys
sys.path.append('/content/drive/My Drive/Progetto Advanced Machine Learning/')
import utils
history_fig = utils.plot_history(trainingMdl)
history_fig.show()

In [0]:
import datetime
save_path = '/content/drive/My Drive/Progetto Advanced Machine Learning/Model Checkpoints/'
save_path += 'Custom Model 1 '
save_path += str(datetime.datetime.now())
model.save(save_path + '.hdf5')

In [91]:
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
history_fig.write_image(save_path +'.pdf')

/usr/local/bin/orca: Text file busy
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgtk2.0-0 is already the newest version (2.24.32-1ubuntu1).
libgconf-2-4 is already the newest version (3.2.6-4ubuntu1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [0]:
import pickle
with open(save_path +'.history.pickle', 'wb') as f:
  pickle.dump(trainingMdl.history, f)

In [95]:
import importlib
importlib.reload(utils)

<module 'utils' from '/content/drive/My Drive/Progetto Advanced Machine Learning/utils.py'>